In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from Evaluation import get_metrics
import time
%load_ext autoreload
%autoreload 2

---
## Set Path

In [2]:
# path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
# path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '../'

## Set params based on top 3 GS results
- ML: Final run for GS iloc 52
- Am_like_ML: Final run for GS iloc 50
- Am_20k_users: Final run for GS iloc 7

In [12]:
file_names = ['am_like_ml']#['ml_1m', 'am_like_ml', 'am_20k_users']

In [13]:
pd.read_pickle(path + 'Logs/GS/' + file_names[0] + '_neumf_gs_log').sort_values('val_rec@10', ascending=False).iloc[:3]

,all_val_rec@10,total_val_rec,val_rec@10,train_time,GMF,MLP,NeuMF,learning_rate,batch_size,layers,reg_layers,reg_mf,nolf,epochs,num_neg,weights_dir,optimizer
50,[],0.254,0.054,158.569508,{'loss': []},{'loss': []},"{'loss': [0.57862, 0.47443, 0.41558, 0.37725, ...",0.00005,512,"[16, 32, 16, 8]","[1e-05, 1e-05, 1e-05, 1e-05]","[0, 0]",8,20,4,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam
155,[],0.212,0.046,151.870936,{'loss': []},{'loss': []},"{'loss': [0.49594, 0.32659, 0.28297, 0.2654, 0...",0.00005,512,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-05, 1e-05]",8,20,8,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam
103,[],0.214,0.046,168.121778,{'loss': []},{'loss': []},"{'loss': [0.43756, 0.29277, 0.26578, 0.26185, ...",0.00005,256,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-06, 1e-06]",8,20,8,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam


In [7]:
NeuMF_ml_1m_params = {
    'learning_rate': 0.0001,
    'batch_size': 512,
    'layers': [16,32,16,8],
    'reg_layers': [0,0,0,0],
    'reg_mf': [0,0],
    'nolf': 8,
    'epochs': 20,
    'sample_size': 0,
    'num_neg': 4,
    'weights_dir': '', #f'../Logs/weights/NeuMF_weights_{file_name}/NeuMF_weights',
    'optimizer':'Adam'
}

In [8]:
pd.read_pickle(path + 'Logs/GS/' + file_names[1] + '_neumf_gs_log').sort_values('val_rec@10', ascending=False).iloc[:5]

,all_val_rec@10,total_val_rec,val_rec@10,train_time,GMF,MLP,NeuMF,learning_rate,batch_size,layers,reg_layers,reg_mf,nolf,epochs,num_neg,weights_dir,optimizer
50,[],0.254,0.054,158.569508,{'loss': []},{'loss': []},"{'loss': [0.57862, 0.47443, 0.41558, 0.37725, ...",0.00005,512,"[16, 32, 16, 8]","[1e-05, 1e-05, 1e-05, 1e-05]","[0, 0]",8,20,4,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam
155,[],0.212,0.046,151.870936,{'loss': []},{'loss': []},"{'loss': [0.49594, 0.32659, 0.28297, 0.2654, 0...",0.00005,512,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-05, 1e-05]",8,20,8,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam
103,[],0.214,0.046,168.121778,{'loss': []},{'loss': []},"{'loss': [0.43756, 0.29277, 0.26578, 0.26185, ...",0.00005,256,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-06, 1e-06]",8,20,8,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam
111,[],0.194,0.044,540.383654,{'loss': []},{'loss': []},"{'loss': [0.36203, 0.28288, 0.27778, 0.27679, ...",0.00010,256,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[0, 0]",8,20,8,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam
107,[],0.198,0.044,170.349026,{'loss': []},{'loss': []},"{'loss': [0.42634, 0.28736, 0.26632, 0.26279, ...",0.00005,256,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-05, 1e-05]",8,20,8,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam


In [10]:
NeuMF_am_like_ml_params = {
    'learning_rate': 0.00005,
    'batch_size': 512,
    'layers': [16,32,16,8],
    'reg_layers': [0.0001, 0.0001, 0.0001, 0.0001],#[0.00001,0.00001,0.00001,0.00001],
    'reg_mf': [0.000001,0.000001],#[0,0],
    'nolf': 8,
    'epochs': 20,
    'sample_size': 0,
    'num_neg': 8,
    'weights_dir': '',#f'../Logs/weights/NeuMF_weights_{file_name}/NeuMF_weights',
    'optimizer':'Adam'
}

In [11]:
pd.read_pickle(path + 'Logs/GS/' + file_names[2] + '_neumf_gs_log').sort_values('val_rec@10', ascending=False).iloc[:3]

,all_val_rec@10,total_val_rec,val_rec@10,train_time,GMF,MLP,NeuMF,learning_rate,batch_size,layers,reg_layers,reg_mf,nolf,epochs,num_neg,weights_dir,optimizer
7,[],0.075,0.017,293.028208,{'loss': []},{'loss': []},"{'loss': [0.60951, 0.46641, 0.37973, 0.35444, ...",0.00005,256,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-06, 1e-06]",8,20,4,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam
67,[],0.073,0.016,155.476554,{'loss': []},{'loss': []},"{'loss': [0.59191, 0.42532, 0.36874, 0.35261, ...",0.00010,512,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-06, 1e-06]",8,20,4,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam
161,[],0.058,0.014,306.694566,{'loss': []},{'loss': []},"{'loss': [0.38299, 0.28976, 0.26635, 0.25525, ...",0.00010,512,"[16, 32, 16, 8]","[1e-06, 1e-06, 1e-06, 1e-06]","[1e-06, 1e-06]",8,20,8,../Logs/weights/NeuMF_weights_0/NeuMF_weights,Adam


In [5]:
NeuMF_am_20k_users_params = {
    'learning_rate': 0.00005,
    'batch_size': 256,
    'layers': [16,32,16,8],
    'reg_layers': [0.0001, 0.0001, 0.0001, 0.0001],
    'reg_mf': [0.000001,0.000001],
    'nolf': 8,
    'epochs': 20,
    'sample_size': 0,#len(train_set_mf),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'weights_dir': '',#f'../Logs/weights/NeuMF_weights_{file_name}/NeuMF_weights',
    'optimizer':'Adam'
}

---
# NeuMF Final Runs

In [14]:
tf.random.set_seed(679596)

In [15]:
data_paths = ['Data/Amazon/'] #['Data/ML/', 'Data/Amazon/', 'Data/Amazon/']
params = [NeuMF_am_like_ml_params] #[NeuMF_ml_1m_params, NeuMF_am_like_ml_params, NeuMF_am_20k_users_params]

n_runs = 10

steps = 5
rank_at = 20

In [16]:
from NCF import NCF
for data_path, file_name, neumf_params in zip(data_paths, file_names, params):
    df = pd.read_pickle(path + data_path + file_name)
    total_items = df.item_id.nunique()
    total_users = df.user_id.nunique()
    
    # Load Train Test Val Split
    train_set_mf = pd.read_pickle(path + data_path + file_name + '_train_mf')
    val_set_mf = pd.read_pickle(path + data_path + file_name + '_val_mf')
    test_set_mf = pd.read_pickle(path + data_path + file_name + '_test_mf')
    
    # Load Samples
    ncf = NCF(total_users, total_items, neumf_params)
    sample_path = path + data_path + 'Samples/' + file_name + '_samples_' + str(neumf_params['num_neg']) + '_neg/'
    samples = ncf.load_samples(sample_path, file_name + '_sample', neumf_params['epochs'])
    
    neumf_params['sample_size'] = len(train_set_mf)
    neumf_params['weights_dir'] = f'../Logs/weights/NeuMF_weights_{file_name}/NeuMF_weights'
    
    for i in range(n_runs):
        print(f'run: {i}')
        ncf = NCF(total_users, total_items, NeuMF_params=neumf_params)
        ncf.build_NeuMF_model()
                                
        s = time.time()
        NeuMF_val_metrics = ncf.train_model(name='NeuMF', samples=samples, train_set=train_set_mf, val_set=val_set_mf, verbose=1, num=0)
        t = time.time() - s
        
        ncf.NeuMF = tf.keras.models.load_model('../Logs/weights/NeuMF_0_best_model.h5')
        ranked_df_neumf = ncf.get_predictions('NeuMF', train_set=train_set_mf, test_set=test_set_mf)
        neumf_test_metrics = get_metrics(ranked_df_neumf, stats=False)
        
        additional_info = {}
        additional_info['all_val_rec@10'] = [val_m['recall'].iloc[2] for val_m in NeuMF_val_metrics]
        additional_info['all_val_ndcg@10'] = [val_m['ndcg'].iloc[2] for val_m in NeuMF_val_metrics]
        additional_info['val_metrics'] = NeuMF_val_metrics
        
        additional_info['metrics'] = neumf_test_metrics
        additional_info['train_time'] = t

        store_path = path + '/Logs/final/' + file_name +  '_neumf_log'
        ncf.store_model('NeuMF', store_path, additional_info)


100% |########################################################################|


run: 0

Fitting NeuMF with parameters:
learning_rate                                                5e-05
batch_size                                                     512
layers                                             [16, 32, 16, 8]
reg_layers                        [0.0001, 0.0001, 0.0001, 0.0001]
reg_mf                                              [1e-06, 1e-06]
nolf                                                             8
epochs                                                          20
sample_size                                                 176012
num_neg                                                          8
weights_dir      ../Logs/weights/NeuMF_weights_am_like_ml/NeuMF...
optimizer                                                     Adam
Name: 0, dtype: object
Epoch: 0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


3094/3094 [==============================] - 15s 5ms/step - loss: 0.4832


KeyboardInterrupt: 

In [ ]:
res_path = path + '/Logs/final/' + file_names[0] + '_neumf_log'
pd.read_pickle(res_path)

# Single runs for clarification

In [38]:
data_path = data_paths[0]
file_name = file_names[0]
neumf_params = params[0]

In [39]:
neumf_params['epochs'] = 20

In [40]:
df = pd.read_pickle(path + data_path + file_name)
total_items = df.item_id.nunique()
total_users = df.user_id.nunique()

# Load Train Test Val Split
train_set_mf = pd.read_pickle(path + data_path + file_name + '_train_mf')
val_set_mf = pd.read_pickle(path + data_path + file_name + '_val_mf')
test_set_mf = pd.read_pickle(path + data_path + file_name + '_test_mf')

In [50]:
from NCF import NCF
ncf = NCF(total_users, total_items, NeuMF_params=neumf_params)
ncf.build_NeuMF_model()

# Load Samples
# sample_path = path + data_path + 'Samples/' + file_name + '_samples_' + str(neumf_params['num_neg']) + '_neg/'
# samples = ncf.load_samples(sample_path, file_name + '_sample', neumf_params['epochs'])
    
# neumf_params['sample_size'] = len(train_set_mf)
# neumf_params['weights_dir'] = f'../Logs/weights/NeuMF_weights_{file_name}/NeuMF_weights'

In [51]:
NeuMF_val_metrics = ncf.train_model(name='NeuMF', samples=samples, train_set=train_set_mf, val_set=val_set_mf, verbose=1, num=1)


Fitting NeuMF with parameters:
learning_rate                                                5e-05
batch_size                                                     512
layers                                             [16, 32, 16, 8]
reg_layers                        [0.0001, 0.0001, 0.0001, 0.0001]
reg_mf                                              [1e-06, 1e-06]
nolf                                                             8
epochs                                                          20
sample_size                                                 176012
num_neg                                                          8
weights_dir      ../Logs/weights/NeuMF_weights_am_like_ml/NeuMF...
optimizer                                                     Adam
Name: 0, dtype: object
Epoch: 0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


3094/3094 [==============================] - 16s 5ms/step - loss: 0.5214
Epoch: 1
3094/3094 [==============================] - 16s 5ms/step - loss: 0.3223
save
Epoch: 2
3094/3094 [==============================] - 16s 5ms/step - loss: 0.2750


KeyboardInterrupt: 

In [49]:
ncf.val_rec_10

[0.046, 0.042, 0.046]

In [ ]:
preds = ncf.get_predictions('NeuMF', train_set_mf, test_set_mf)
metrics = get_metrics(preds)
metrics

In [ ]:
ncf2 = NCF(total_users, total_items, NeuMF_params=neumf_params)
ncf2.NeuMF = tf.keras.model.load('../Logs/weights/NeuMF_1_best_model.h5)
best_preds = ncf.get_predictions('NeuMF', train_set_mf, test_set_mf)
best_metrics = get_metrics(best_preds)
best_metrics

In [54]:
NeuMF_val_metrics = ncf.train_model(name='NeuMF', samples=samples, train_set=train_set_mf, val_set=val_set_mf, verbose=1)


Fitting NeuMF with parameters:
learning_rate                               5e-05
batch_size                                    512
layers                            [16, 32, 16, 8]
reg_layers       [0.0001, 0.0001, 0.0001, 0.0001]
reg_mf                             [1e-06, 1e-06]
nolf                                            8
epochs                                         20
sample_size                                     0
num_neg                                         8
weights_dir                                      
optimizer                                    Adam
Name: 0, dtype: object
Epoch: 0
3094/3094 [==============================] - 14s 5ms/step - loss: 0.5234
[   rank_at  hitcounts  recall  precision      ndcg
0        1         10   0.020   0.020000  0.020000
1        5         21   0.042   0.008400  0.032032
2       10         22   0.044   0.004400  0.032699
3       15         23   0.046   0.003067  0.033211
4       20         23   0.046   0.002300  0.033211]
Epoch

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
ncf.NeuMF.load_weights(f'../Logs/weights/NeuMF_weights_{file_name}/NeuMF_weights').expect_partial()

In [ ]:
sample_preds = ncf.sample_prediction('NeuMF', train_set_mf, test_set_mf, exclude_already_seen=False)
get_metrics(sample_preds)